In [ ]:
from syft_rds.orchestra import setup_rds_server, remove_rds_stack_dir

In [ ]:
remove_rds_stack_dir(key="rds_stack", root_dir="./")

## DO1 Uploads Dataset

In [ ]:
do_stack_1 = setup_rds_server(email="do1@openmined.org", key="rds_stack", root_dir="./")

In [ ]:
do_client_1 = do_stack_1.init_session(host="do1@openmined.org")

In [ ]:
from utils import generate_crop_data, create_readme

DATASET_FILE_NAME = "crop_stock_data.csv"
DATASET_PATH = "datasets/part_2"
DATASET_PRIVATE_PATH = f"{DATASET_PATH}/private"
DATASET_MOCK_PATH = f"{DATASET_PATH}/mock"

In [ ]:
# Private Data
generate_crop_data(
    num_rows=10, output_path=f"{DATASET_PRIVATE_PATH}/{DATASET_FILE_NAME}"
)

In [ ]:
# Mock Data
generate_crop_data(num_rows=10, output_path=f"{DATASET_MOCK_PATH}/{DATASET_FILE_NAME}")

In [ ]:
README_PATH = f"{DATASET_PATH}/README.md"
create_readme(output_path=README_PATH)

In [ ]:
from syft_notebook_ui import show_dir
from pathlib import Path

# Now we can see the dataset is in our sync folder
show_dir(Path("datasets/part_2"))

In [ ]:
try:
    dataset = do_client_1.dataset.create(
        name="Organic Crop Stock Data",
        summary="This dataset contains information about organic crop stock.",
        readme_path=README_PATH,
        private_path=DATASET_PRIVATE_PATH,
        mock_path=DATASET_MOCK_PATH,
        # auto_approval = ["ds@openmined.org"]
    )
    dataset.describe()
except Exception as e:
    print(f"Error: {e}")

## DO2 Upload dataset

In [ ]:
do_stack_2 = setup_rds_server(email="do2@openmined.org", key="rds_stack", root_dir=".")

In [ ]:
do_client_2 = do_stack_2.init_session(host="do2@openmined.org")

In [ ]:
DATASET_FILE_NAME = "crop_stock_data.csv"
DATASET_PATH = "datasets/part_1"
DATASET_PRIVATE_PATH = f"{DATASET_PATH}/private"
DATASET_MOCK_PATH = f"{DATASET_PATH}/mock"

In [ ]:
# Private Data
generate_crop_data(
    num_rows=10, output_path=f"{DATASET_PRIVATE_PATH}/{DATASET_FILE_NAME}"
)

In [ ]:
# Mock Data
generate_crop_data(num_rows=10, output_path=f"{DATASET_MOCK_PATH}/{DATASET_FILE_NAME}")

In [ ]:
try:
    dataset = do_client_2.dataset.create(
        name="Organic Crop Stock Data",
        summary="This dataset contains information about organic crop stock.",
        readme_path=README_PATH,
        private_path=DATASET_PRIVATE_PATH,
        mock_path=DATASET_MOCK_PATH,
        # auto_approval = ["ds@openmined.org"]
    )
    dataset.describe()
except Exception as e:
    print(f"Error: {e}")

## DS submits jobs

In [ ]:
ds_stack = setup_rds_server(email="ds@openmined.org", key="rds_stack", root_dir=".")

In [ ]:
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"

do_client_1 = ds_stack.init_session(host=DO1)
print("Logged into: ", do_client_1.host)

do_client_2 = ds_stack.init_session(host=DO2)
print("Logged into: ", do_client_2.host)

assert not do_client_1.is_admin
assert not do_client_2.is_admin

In [ ]:
DATASET_NAME = "Organic Crop Stock Data"

dataset1 = do_client_1.dataset.get(name=DATASET_NAME)
dataset1.describe()

In [ ]:
dataset2 = do_client_2.dataset.get(name=DATASET_NAME)
dataset2.describe()

In [ ]:
do_client_1.runtime.get_all()

In [ ]:
CODE_DIR = "./code"
Path(CODE_DIR).mkdir(parents=True, exist_ok=True)

script_content = """import os
from pathlib import Path
from sys import exit

import pandas as pd

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

dataset_paths = [ Path(dataset_path) for dataset_path in DATA_DIR.split(",")]
total_carrots = 0
total_tomatoes = 0

for dataset_path in dataset_paths:
    if not dataset_path.exists():
        print("Warning: Dataset path does not exist:", dataset_path)
        exit(1)
    df = pd.read_csv(dataset_path / "crop_stock_data.csv")
    total_carrots += df[df["Product name"] == "Carrots"]["Quantity"].sum()
    total_tomatoes += df[df["Product name"] == "Tomatoes"]["Quantity"].sum()

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write(f"Total Carrots: {total_carrots}\n")
    f.write(f"Total Tomatoes: {total_tomatoes}\n")
"""


# Write the script to the entrypoint file
entrypoint_path = Path(CODE_DIR) / "entrypoint.py"
with open(entrypoint_path, "w") as f:
    f.write(script_content)

print(f"Successfully wrote entrypoint script to: {entrypoint_path}")

In [ ]:
datasites = [do_client_1, do_client_2]

for client in datasites:
    job = client.job.submit(
        name="Crop Avg Experiment",
        description="Farming Coop Avg Experiment",
        user_code_path="./code",
        dataset_name=DATASET_NAME,
        tags=["enclave", "syft"],
        entrypoint="entrypoint.py",
    )
    print(job)